In [1]:
import sqlite3
import datetime

# Підключення до бази даних
conn = sqlite3.connect('logistics.db')
cursor = conn.cursor()

# Створення таблиць
cursor.execute('''
CREATE TABLE IF NOT EXISTS Warehouses (
    warehouse_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    location TEXT,
    capacity INTEGER
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Resources (
    resource_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    description TEXT,
    quantity INTEGER
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS ResourceMovements (
    movement_id INTEGER PRIMARY KEY AUTOINCREMENT,
    resource_id INTEGER,
    from_warehouse_id INTEGER,
    to_warehouse_id INTEGER,
    quantity INTEGER,
    date TEXT,
    FOREIGN KEY(resource_id) REFERENCES Resources(resource_id),
    FOREIGN KEY(from_warehouse_id) REFERENCES Warehouses(warehouse_id),
    FOREIGN KEY(to_warehouse_id) REFERENCES Warehouses(warehouse_id)
);
''')

# Додавання тестових даних
cursor.execute('''
INSERT INTO Warehouses (name, location, capacity) 
VALUES ('Склад 1', 'Львів', 1000), 
       ('Склад 2', 'Київ', 800);
''')

cursor.execute('''
INSERT INTO Resources (name, description, quantity) 
VALUES ('Товари 1', 'Опис товару 1', 500), 
       ('Товари 2', 'Опис товару 2', 300);
''')

conn.commit()

# Функція для переміщення ресурсів
def move_resources(resource_id, from_warehouse_id, to_warehouse_id, quantity):
    cursor.execute('SELECT quantity FROM Resources WHERE resource_id = ?', (resource_id,))
    current_quantity = cursor.fetchone()[0]
    
    if current_quantity < quantity:
        print(f"Недостатньо ресурсів для переміщення. Доступно тільки {current_quantity} одиниць.")
        return
    
    cursor.execute('UPDATE Resources SET quantity = quantity - ? WHERE resource_id = ?', (quantity, resource_id))
    cursor.execute('UPDATE Resources SET quantity = quantity + ? WHERE resource_id = ?', (quantity, resource_id))
    cursor.execute('INSERT INTO ResourceMovements (resource_id, from_warehouse_id, to_warehouse_id, quantity, date) VALUES (?, ?, ?, ?, ?)', 
                   (resource_id, from_warehouse_id, to_warehouse_id, quantity, datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')))
    conn.commit()

    print(f"Переміщення {quantity} одиниць ресурсу {resource_id} з складу {from_warehouse_id} на склад {to_warehouse_id}.")

# Приклад використання функції
move_resources(1, 1, 2, 100)

# Перевірка результатів
cursor.execute('SELECT * FROM Resources')
resources = cursor.fetchall()
print("Ресурси на складах:")
for resource in resources:
    print(resource)

cursor.execute('SELECT * FROM ResourceMovements')
movements = cursor.fetchall()
print("\nІсторія переміщень:")
for movement in movements:
    print(movement)

# Закриття з'єднання
conn.close()


Переміщення 100 одиниць ресурсу 1 з складу 1 на склад 2.
Ресурси на складах:
(1, 'Товари 1', 'Опис товару 1', 500)
(2, 'Товари 2', 'Опис товару 2', 300)

Історія переміщень:
(1, 1, 1, 2, 100, '2025-01-25 09:57:29')
